In [1]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 92.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 153MB/s]


In [2]:
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [25]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [5]:
train_df.head()

,confidence,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,clean_text,target
0,1.000000,3,5.311000,89,63.33,10.84,98.92,3.83,17.80,12.36,...,1.0,1.74515,1.42143,1.83704,0.58,27,6.376667,0.052071,reluctant start one really ashamed ask help on...,1
1,1.000000,3,4.220972,108,19.76,11.45,95.96,1.00,15.43,12.96,...,1.0,1.71501,1.57333,1.85046,1.00,0,5.593690,-0.022917,severe addict overdose young addictionalcoholi...,1
2,0.500000,15,5.548481,79,12.35,44.98,69.44,1.00,15.80,16.46,...,1.0,1.80983,1.48267,1.86705,0.91,5,6.703646,-0.460000,feel sad feel sad pray get help desperately ne...,1
3,0.833333,18,4.352143,81,93.97,40.24,99.00,25.77,16.20,12.35,...,1.0,1.70892,1.84444,1.81327,0.85,11,5.150095,0.048636,sleep car month month back foot stay parking s...,1
4,0.833333,4,7.366712,135,40.97,72.32,75.74,1.00,27.00,12.59,...,1.0,1.75929,1.58319,1.84228,1.00,5,7.778319,0.031250,speed try catch continue yell face girl turn a...,0


In [6]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [9]:
path = "/content/drive/MyDrive/NLP/Stress Detection/fasttext-wiki-news-subwords-300.gz"

In [10]:
model_fasttext = gensim.models.KeyedVectors.load_word2vec_format(path)

# Any Confidence

In [26]:
X_train_any = train_df['clean_text'].to_numpy()
y_train_any = train_df['target'].to_numpy()
X_test_any = test_df['clean_text'].to_numpy()
y_test_any = test_df['target'].to_numpy()

## Models

In [27]:
# Tokenize text data
tokenizer = Tokenizer(num_words=7000)
tokenizer.fit_on_texts(np.concatenate((X_train_any, X_test_any), axis=0))
X_train_sequences = tokenizer.texts_to_sequences(X_train_any)
X_test_sequences = tokenizer.texts_to_sequences(X_test_any)

In [28]:
high = 0
for temp in X_train_sequences:
  high = max(high, len(temp))
high

125

In [20]:
# Pad sequences to same length
max_len = 150
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, maxlen=max_len)

In [21]:
# Create embedding layer
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model_fasttext:
        embedding_matrix[i] = model_fasttext[word]
embedding_layer = Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

### ANN

In [ ]:
model_dense = Sequential()
model_dense.add(embedding_layer)
model_dense.add(GlobalAveragePooling1D())
model_dense.add(Dense(64, activation='relu'))
model_dense.add(Dropout(0.3))
# model_dense.add(Dense(32, activation='relu'))
model_dense.add(Dense(1, activation='sigmoid'))

In [ ]:
model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
model_dense.fit(X_train_padded_sequences, y_train_any, epochs=100, batch_size=32, verbose=1, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test_any))

Epoch 1/100
89/89 [==============================] - 1s 8ms/step - loss: 0.6885 - accuracy: 0.5408 - val_loss: 0.6901 - val_accuracy: 0.4852
Epoch 2/100
89/89 [==============================] - 1s 7ms/step - loss: 0.6806 - accuracy: 0.5524 - val_loss: 0.6801 - val_accuracy: 0.5190
Epoch 3/100
89/89 [==============================] - 1s 6ms/step - loss: 0.6705 - accuracy: 0.6031 - val_loss: 0.6631 - val_accuracy: 0.7117
Epoch 4/100
89/89 [==============================] - 1s 7ms/step - loss: 0.6570 - accuracy: 0.6626 - val_loss: 0.6495 - val_accuracy: 0.6582
Epoch 5/100
89/89 [==============================] - 1s 7ms/step - loss: 0.6405 - accuracy: 0.6858 - val_loss: 0.6300 - val_accuracy: 0.6990
Epoch 6/100
89/89 [==============================] - 1s 6ms/step - loss: 0.6235 - accuracy: 0.7020 - val_loss: 0.6272 - val_accuracy: 0.6470
Epoch 7/100
89/89 [==============================] - 1s 7ms/step - loss: 0.6122 - accuracy: 0.7023 - val_loss: 0.6080 - val_accuracy: 0.6751
Epoch 8/100
8

In [ ]:
y_pred_dense = model_dense.predict(X_test_padded_sequences)
y_pred_dense = (y_pred_dense > 0.5).astype(int)
get_report(y_test_any, y_pred_dense)

23/23 [==============================] - 0s 4ms/step


{'Accuracy': 0.7637130801687764,
 'Precision': 0.7322834645669292,
 'Recall': 0.808695652173913,
 'F1-Score': 0.768595041322314}

In [ ]:
model_dense.save('fasttext_any.keras')

In [29]:
model_any = tf.keras.models.load_model("/content/drive/MyDrive/NLP/Stress Detection/Best_Models/Only Text/only_text_fasttext_any.keras")

In [33]:
y_pred_dense = model_any.predict(X_test_padded_sequences)
y_pred_dense = (y_pred_dense > 0.5).astype(int)
print(confusion_matrix(y_test_any, y_pred_dense))

23/23 [==============================] - 0s 8ms/step
[[264 102]
 [ 66 279]]


In [34]:
print(classification_report(y_test_any, y_pred_dense))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       366
           1       0.73      0.81      0.77       345

    accuracy                           0.76       711
   macro avg       0.77      0.77      0.76       711
weighted avg       0.77      0.76      0.76       711



# 80% Confidence

In [35]:
train_df_80 = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [36]:
test_df_80 = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)

In [37]:
X_train_80 = train_df_80['clean_text'].to_numpy()
y_train_80 = train_df_80['target'].to_numpy()

In [38]:
X_test_80 = test_df_80['clean_text'].to_numpy()
y_test_80 = test_df_80['target'].to_numpy()

## Models

In [47]:
# Tokenize text data
tokenizer = Tokenizer(num_words=7000)
tokenizer.fit_on_texts(np.concatenate((X_train_80, X_test_80), axis=0))
X_train_sequences = tokenizer.texts_to_sequences(X_train_80)
X_test_sequences = tokenizer.texts_to_sequences(X_test_80)

In [48]:
high = 0
for temp in X_train_sequences:
  high = max(high, len(temp))
high

141

In [49]:
# Pad sequences to same length
max_len = 150
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, maxlen=max_len)

In [ ]:
# Create embedding layer
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model_fasttext:
        embedding_matrix[i] = model_fasttext[word]
embedding_layer = Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

### ANN

In [ ]:
model_dense_80 = Sequential()
model_dense_80.add(embedding_layer)
model_dense_80.add(GlobalAveragePooling1D())
model_dense_80.add(Dense(64, activation='relu'))
model_dense_80.add(Dropout(0.4))
model_dense_80.add(Dense(32, activation='relu'))
model_dense_80.add(Dense(1, activation='sigmoid'))
model_dense_80.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
model_dense_80.fit(X_train_padded_sequences, y_train_80, epochs=100, batch_size=32, verbose=1, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test_80))

Epoch 1/100
58/58 [==============================] - 2s 16ms/step - loss: 0.6850 - accuracy: 0.5464 - val_loss: 0.6871 - val_accuracy: 0.5161
Epoch 2/100
58/58 [==============================] - 1s 10ms/step - loss: 0.6693 - accuracy: 0.5655 - val_loss: 0.6631 - val_accuracy: 0.5355
Epoch 3/100
58/58 [==============================] - 1s 9ms/step - loss: 0.6322 - accuracy: 0.6769 - val_loss: 0.6081 - val_accuracy: 0.7161
Epoch 4/100
58/58 [==============================] - 0s 6ms/step - loss: 0.5734 - accuracy: 0.7385 - val_loss: 0.5472 - val_accuracy: 0.7677
Epoch 5/100
58/58 [==============================] - 0s 6ms/step - loss: 0.5205 - accuracy: 0.7598 - val_loss: 0.5054 - val_accuracy: 0.7677
Epoch 6/100
58/58 [==============================] - 0s 7ms/step - loss: 0.4770 - accuracy: 0.7898 - val_loss: 0.5031 - val_accuracy: 0.7505
Epoch 7/100
58/58 [==============================] - 0s 6ms/step - loss: 0.4607 - accuracy: 0.7926 - val_loss: 0.4681 - val_accuracy: 0.7892
Epoch 8/100

In [ ]:
y_pred_dense = model_dense_80.predict(X_test_padded_sequences)
y_pred_dense = (y_pred_dense > 0.5).astype(int)
get_report(y_test_80, y_pred_dense)

15/15 [==============================] - 0s 4ms/step


{'Accuracy': 0.810752688172043,
 'Precision': 0.7945736434108527,
 'Recall': 0.8541666666666666,
 'F1-Score': 0.823293172690763}

In [ ]:
model_dense_80.save('fasttext_80.keras')

In [43]:
model_80 = tf.keras.models.load_model("/content/drive/MyDrive/NLP/Stress Detection/Best_Models/Only Text/only_text_fasttext_80.keras")

In [51]:
y_pred_dense = model_80.predict(X_test_padded_sequences)
y_pred_dense = (y_pred_dense > 0.5).astype(int)
print(confusion_matrix(y_test_80, y_pred_dense))

15/15 [==============================] - 0s 9ms/step
[[172  53]
 [ 35 205]]


In [52]:
print(classification_report(y_test_80, y_pred_dense))

              precision    recall  f1-score   support

           0       0.83      0.76      0.80       225
           1       0.79      0.85      0.82       240

    accuracy                           0.81       465
   macro avg       0.81      0.81      0.81       465
weighted avg       0.81      0.81      0.81       465

